# EC2202 Graphs

**Disclaimer.**
This code examples are based on 
1. [LeetCode](https://leetcode.com/)
2. [GeeksForGeeks](https://practice.geeksforgeeks.org/)
3. Coding Interviews

In [ ]:
import doctest

## Disjoint Sets

### Quick Find

In [ ]:
class DisjointSets:
  """
  >>> ds = DisjointSets(10)
  >>> ds.union(1, 2)
  >>> ds.union(2, 5)
  >>> ds.union(5, 6)
  >>> ds.union(6, 7)
  >>> ds.union(3, 8)
  >>> ds.union(8, 9)
  >>> ds.connected(1, 5)
  True
  >>> ds.connected(5, 7))
  True
  >>> ds.connected(4, 9)
  False
  >>> ds.union(9, 4)
  >>> ds.connected(4, 9)
  True
  """
  def __init__(self, size):
    self.root = [i for i in range(size)]

  def find(self, x):  # returns the root of x; O(1)
    return self.root[x]
  
  def connected(self, x, y):  # O(1)
    return self.find(x) == self.find(y)
  
  # 'ppp' exercise
  def union(self, x, y):  # O(N)
    # check the roots of x and y
    # if they do not match, update the roots
    root_x = self.find(x)  # self.root[x]
    root_y = self.find(y)
    if root_x != root_y:
      for i in range(len(self.root)):
        if self.root[i] == root_y:
          self.root[i] = root_x

In [ ]:
# Test Case
ds = DisjointSets(10)
# 1-2-5-6-7 3-8-9 4
ds.union(1, 2)
ds.union(2, 5)
ds.union(5, 6)
ds.union(6, 7)
ds.union(3, 8)
ds.union(8, 9)
print(ds.connected(1, 5))  # true
print(ds.connected(5, 7))  # true
print(ds.connected(4, 9))  # false
# 1-2-5-6-7 3-8-9-4
ds.union(9, 4)
print(ds.connected(4, 9))  # true

### Quick Union

In [ ]:
class DisjointSets:
  def __init__(self, size):
    self.root = [i for i in range(size)]

  # 'ppp' exercise
  def find(self, x):  # < O(N)
    while x != self.root[x]:
      x = self.root[x]
    return x
  
  # 'ppp' exercise
  def union(self, x, y):  # < O(N)
    root_x = self.find(x)
    root_y = self.find(y)
    if root_x != root_y:
      self.root[root_y] = root_x

  def connected(self, x, y):  # < O(N)
    return self.find(x) == self.find(y)

In [ ]:
# Test Case
ds = DisjointSets(10)
# 1-2-5-6-7 3-8-9 4
ds.union(1, 2)
ds.union(2, 5)
ds.union(5, 6)
ds.union(6, 7)
ds.union(3, 8)
ds.union(8, 9)
print(ds.connected(1, 5))  # true
print(ds.connected(5, 7))  # true
print(ds.connected(4, 9))  # false
# 1-2-5-6-7 3-8-9-4
ds.union(9, 4)
print(ds.connected(4, 9))  # true

### Union by rank

We have implemented two kinds of “disjoint sets” so far, and they both have a concerning inefficiency. Specifically, the quick find implementation will always spend O(n) time on the union operation and in the quick union implementation, it is possible for all the vertices to form a line after connecting them using union, which results in the worst-case scenario for the find function. Is there any way to optimize these implementations?

Of course, there is; it is to union by rank. The word “rank” means ordering by specific criteria. Previously, for the union function, we always chose the root node of x and set it as the new root node for the other vertex. However, by choosing the parent node based on certain criteria (by rank), we can limit the maximum height of each vertex.

To be specific, the “rank” refers to the height of each vertex. When we union two vertices, instead of always picking the root of x (or y, it doesn't matter as long as we're consistent) as the new root node, we choose the root node of the vertex with a larger “rank”. We will merge the shorter tree under the taller tree and assign the root node of the taller tree as the root node for both vertices. In this way, we effectively avoid the possibility of connecting all vertices into a straight line. This optimization is called the “disjoint set” with union by rank.

In [ ]:
class DisjointSets:
  def __init__(self, size):
    self.root = [i for i in range(size)]
    self.rank = [1] * size

  def find(self, x):  # O(log N)
    while x != self.root[x]:
      x = self.root[x]
    return x
  
  def connected(self, x, y):  # O(log N)
    return self.find(x) == self.find(y)

  def union(self, x, y):  # O(log N)
    root_x = self.find(x)
    root_y = self.find(y)
    if root_x != root_y:
      if self.rank[root_x] > self.rank[root_y]:
        self.root[root_y] = root_x
      elif self.rank[root_x] < self.rank[root_y]:
        self.root[root_x] = root_y
      else:  # x and y have the same rank
        self.root[root_y] = root_x
        self.rank[root_x] += 1

In [ ]:
# Test Case
ds = DisjointSets(10)
# 1-2-5-6-7 3-8-9 4
ds.union(1, 2)
ds.union(2, 5)
ds.union(5, 6)
ds.union(6, 7)
ds.union(3, 8)
ds.union(8, 9)
print(ds.connected(1, 5))  # true
print(ds.connected(5, 7))  # true
print(ds.connected(4, 9))  # false
# 1-2-5-6-7 3-8-9-4
ds.union(9, 4)
print(ds.connected(4, 9))  # true

### Path compression

In the previous implementation of the “disjoint set”, notice that to find the root node, we need to traverse the parent nodes sequentially until we reach the root node. If we search the root node of the same element again, we repeat the same operations. Is there any way to optimize this process?

The answer is yes! After finding the root node, we can update the parent node of all traversed elements to their root node. When we search for the root node of the same element again, we only need to traverse two elements to find its root node, which is highly efficient. So, how could we efficiently update the parent nodes of all traversed elements to the root node? The answer is to use “recursion”. This optimization is called “path compression”, which optimizes the find function.

In [ ]:
class DisjointSets:
  def __init__(self, size):
    self.root = [i for i in range(size)]

  # 'ppp' exercise
  # 5 <- 4 <- 3 <- 2 <- 1 (linked list > tree)
  # ind : [1, 2, 3, 4, 5]
  # root: [2, 3, 4, 5, 5]
  # find: [5, 5, 5, 5, 5]
  def find(self, x):
    if x == self.root[x]:
      return x
    self.root[x] = self.find(self.root[x])
    return self.root[x]
  
  def union(self, x, y):
    root_x = self.find(x)
    root_y = self.find(y)
    if root_x != root_y:
      self.root[root_y] = root_x

  def connected(self, x, y):
    return self.find(x) == self.find(y)

### Combination of path compression and union by rank

In [ ]:
class DisjointSets:
  def __init__(self, size):
    self.root = [i for i in range(size)]
    # Use a rank array to record the height of each vertex,
    # i.e., the "rank" of each vertex.
    # The initial "rank" of each vertex is 1, because each of them is
    # a standalone vertex with no connection to other vertices.
    self.rank = [1] * size

  # The find function here is the same as
  # that in the disjoint set with path compression.
  def find(self, x):
    if x == self.root[x]:
      return x
    self.root[x] = self.find(self.root[x])
    return self.root[x]

  # The union function with union by rank
  def union(self, x, y):
    root_x = self.find(x)
    root_y = self.find(y)
    if root_x != root_y:
      if self.rank[root_x] > self.rank[root_y]:
        self.root[root_y] = root_x
      elif self.rank[root_x] < self.rank[root_y]:
        self.root[root_x] = root_y
      else:
        self.root[root_y] = root_x
        self.rank[root_x] += 1

  def connected(self, x, y):
    return self.find(x) == self.find(y)

class Disjoint sets
1. find -> Quick find
2. union -> Quick union
3. connected

성능개선법
1. Union by rank
2. path compression
3. 1번 + 2번

### 'ppp' exercise

There are n people in a social group labeled from 0 to n - 1. You are given an array logs where logs[i] = [timestamp_i, x_i, y_i] indicates that x_i and y_i will be friends at the time timestamp_i.

Friendship is symmetric. That means if a is friends with b, then b is friends with a. Also, person a is acquainted with a person b if a is friends with b, or a is a friend of someone acquainted with b.

Return the earliest time for which every person became acquainted with every other person. If there is no such earliest time, return -1.

In [ ]:
def earliest_friends(logs, n):
  """
  >>> logs = [
  ...    [20190101,0,1],[20190104,3,4],[20190107,2,3],[20190211,1,5],
  ...    [20190224,2,4],[20190301,0,3],[20190312,1,2],[20190322,4,5]
  ...  ]
  >>> n = 6
  >>> earliest_friends(logs, n)
  20190301

  >>> logs = [[0,2,0],[1,0,1],[3,0,3],[4,1,2],[7,3,1]]
  >>> n = 4
  >>> earliest_friends(logs, n)
  3
  """


  
  return -1